# Machine Learning
En este notebook se aplicarán los distintos modelos, algoritmos o métricas de machine learning que nos permitan crear nuestros algoritmos de recomendación:

item a item:
```
def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.
```

En un futuro se implementará el algoritmo de usuario a item:

```
def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.
```

# 1. Importamos las librerías necesarias

In [142]:
import pandas as pd # Con pandas trabajaremos los datos.
from sklearn.metrics.pairwise import cosine_similarity # Con el objeto cosine_similarity de sklearn, podremos saber la similitud que tiene un item con otro, útil para recomendaciones similares.

# 2. Cargamos los datos

Leemos el dataframe que creamos previamente en el EDA y lo previsualizamos para verificar que sea el correcto.

In [143]:
df = pd.read_csv('games_ML.csv')

In [144]:
df.head(3)

,id,name,1,2,3,4,5,6,7,8,9,10
0,761140,Lost Summoner Kitty,-0.663424,-0.394529,-0.402296,0.307438,0.771669,-0.544150,0.039934,-0.064927,-0.255702,0.140794
1,643980,Ironbound,-0.352296,0.314932,0.899000,1.389335,0.086969,0.263993,-0.681269,0.650172,0.039089,-0.062176
2,670290,Real Pool 3D - Poolians,-0.688840,0.202145,-0.370959,0.441656,1.035726,-0.533025,0.172840,0.696757,0.323371,0.726916


# 3. Usamos la similitud de coseno para obtener la similaridad

Aplicamos la similitud del coseno a los valores del dataframe, no a el id y el nombre

In [145]:
# Esto nos crea una matriz de numpy con similitudes de los items.
cos = cosine_similarity(df.drop(columns=['name', 'id']))
# Creamos un dataframe con ids y nombres para identificarlos más adelante y tener una salida de DataFrame más organizada.
df_ids = df[['id', 'name']].set_index('id')
# Creamos el DataFrame de la similitud del coseno, siendo las columnas el nombre del producto con el que se compara, y el indice el id del producto comparado.
df_cos = pd.DataFrame(cos, columns=df['name'], index=df['id'])

Previsualizamos el resultado

In [146]:
df_cos.head()

name,Lost Summoner Kitty,Ironbound,Real Pool 3D - Poolians,弹炸人2222,Battle Royale Trainer,SNOW - All Access Basic Pass,SNOW - All Access Pro Pass,SNOW - All Access Legend Pass,Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition,Beach Rules,...,Counter-Strike: Condition Zero,Agent X: Equation Rider,Snail Trek - Chapter 3: Lettuce Be,Raining blocks,Bravium,Kebab it Up!,Colony On Mars,LOGistICAL: South Africa,Russian Roads,EXIT 2 - Directions
id,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.003810,0.698632,0.231619,-0.046687,-0.284276,-0.284276,-0.284276,0.213597,0.843852,...,0.159739,0.742512,0.277338,0.431268,0.328466,0.115726,0.787453,0.300001,0.349373,0.451817
643980,0.003810,1.000000,0.230950,-0.404121,-0.486018,0.262310,0.262310,0.262310,-0.081481,-0.143038,...,-0.066986,-0.103362,-0.132711,-0.355745,0.048201,-0.304579,0.150137,0.071571,0.177094,-0.165342
670290,0.698632,0.230950,1.000000,0.171412,-0.031541,0.187424,0.187424,0.187424,-0.054223,0.489030,...,0.177569,0.240483,-0.138043,0.229901,-0.327324,-0.140170,0.546315,0.002560,0.226737,0.079599
767400,0.231619,-0.404121,0.171412,1.000000,0.525308,-0.696890,-0.696890,-0.696890,0.597485,0.564201,...,0.476913,0.386613,0.459726,0.156882,-0.159372,0.524153,-0.088481,-0.171861,-0.328126,0.220566
772540,-0.046687,-0.486018,-0.031541,0.525308,1.000000,-0.312767,-0.312767,-0.312767,0.165265,-0.029781,...,0.366806,0.222387,0.461141,-0.219178,-0.155575,0.245206,-0.244818,-0.427990,0.150584,-0.152748


Podemos observar que el ID `761140` correspondiente al nombre `Lost Summoner Kitty` son idénticos (1) ya que es el mismo producto. Esto se tendrá en cuenta más adelante para que al extraer los 5 más similares, ese no sea uno de ellos.

---

# 4. Obtenemos los N productos más similares al ID

## 4.1 Función para obtener los similares

Creamos una función que obtenga los productos más similares dentro del dataframe previamente creado de la similitud del coseno.

In [147]:
def obtener_top_similares(df_similitud, id, df_ids=None, nombre=None):
    '''
    Top 5 similares a ID.

    Busca y retorna los 5 nombres de juegos más similares al ID de juego proporcionado.

    Requiere:
    Pandas
    
    Recibe:
    Pandas DataFrame(df_similitud): Un dataframe de la similitud del coseno con indices como IDS y con columnas como NOMBRES.
    int(id): Un id.
    Pandas Dataframe(): Un dataframe o Series de pandas, con IDS y NOMBRES.
    String(Nombre) (opcional): Si ya se tiene el nombre, se puede ingresar directamente para no buscarlo en el dataframe.

    Retorna:
    Una serie de pandas con los 5 nombres de los juegos más parecidos al juego del ID ingresado.
    '''
    # Obtenemos el nombre del producto del dataframe que contiene los nombres con los ID. Esto solo si no se proporcionó un nombre.
    if not nombre:
        nombre = df_ids.loc[id].iloc[0]
    # Buscamos la fila del ID en el dataframe de la similitud del coseno.
    fila_id = df_similitud.loc[id]
    # Si esta vacío, es porque no existe el id. Retornamos un string para guiarnos más adelante si esto llega a ocurrir.
    if fila_id.empty:
        return "ID no encontrado"
    # Ordena los valores de similitud en orden descendente, eliminando el nombre del id ingresado para no tener redundancias.
    fila_ordenada = fila_id.drop(nombre).sort_values(ascending=False)

    # Tomamos los 5 primeros valores de la lista ordenada.
    top_5_similares = fila_ordenada.head(5)
    # Retornamos esos 5 valores.
    return top_5_similares

## 4.2 Usamos la función para crear un dataframe de recomendaciones

Creamos un dataframe de recomendaciones en base a cada producto, esto con el fin de que la API solo deba consultar las recomendaciones, mas no crearlas.

In [148]:
# Almacenamos los datos en un diccionario para volverlo un dataframe después.
dic = {'id':[], 'name':[], 'recommend':[]}
# Iteramos en cada indice y nombre de nuestro DataFrame de IDS.
for i, name in df_ids.iterrows():
    # Extraemos el nombre del producto que evaluaremos.
    name = name.iloc[0]
    # Agregamos el nombre y el id extraídos al diccionario.
    dic['id'].append(i)
    dic['name'].append(name)
    # Obtenemos los productos similares, solo nos interesan sun indices, que en este caso son sus nombres, sus valores son el puntaje de similitud que el algoritmo les asignó.
    # Para esto usamos la función top_similares, le pasamos el dataframe de similitud del coseno, junto con un indice, y un nombre. Consultar la función para más información sobre su funcionamiento y los parámetros.
    similares = obtener_top_similares(df_cos, i, nombre=name).index
    # Agregamos a los recomendados, la lista de los recomendados.
    dic['recommend'].append(similares.to_list())

In [149]:
# Convertimos el diccionario a un DataFrame.
df_recommend = pd.DataFrame(dic)

In [150]:
# Visualizamos el DataFrame.
df_recommend.head()

,id,name,recommend
0,761140,Lost Summoner Kitty,"[Wooden Battles, Infect and Destroy, Hunters L..."
1,643980,Ironbound,"[Immortal Empire, Elemental Heroes, Heroes of ..."
2,670290,Real Pool 3D - Poolians,"[Billiards, Kitchen Simulator 2, Snooker-onlin..."
3,767400,弹炸人2222,"[Biozone, CHASER, Turtle Odyssey, Luxor: 5th P..."
4,772540,Battle Royale Trainer,"[Cargo! The Quest for Gravity, The jungle, Cyb..."


# 5. Exportamos los datos

In [151]:
df_recommend.to_csv('../Api/ApiData/recommend_item_to_item.csv', index=False)